In [4]:
from openai.lib.azure import AzureOpenAI
from dotenv import load_dotenv
import os
import json
load_dotenv(".env")

api_base = os.environ["API_BASE"]
api_key = os.environ["API_KEY"]
deployment_name = os.environ["DEPLOYMENT_NAME"]
api_version = os.environ["API_VERSION"]

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

In [5]:
import os
import fitz
import base64

def convert_pdf_to_images(pdf_path, output_folder):
    image_name = f"{os.path.basename(pdf_path)}.png"
    image_path = os.path.join(output_folder, image_name)
    image_base64 = []
    with fitz.open(pdf_path) as doc:
        page = doc[0]
        pix = page.get_pixmap()
        pix.save(image_path)
    image_base64.append(base64.b64encode(open(image_path, "rb").read()).decode('utf-8'))
    os.remove(image_path)
    return image_base64

In [6]:
def get_image_details(images, prompt: str):
    messages = [
        {"role": "system", "content": '''You are a helpful assistant in document analysis.and answer in json format ex:{
            document_type: "invoice",
            }, never include extra '```json\n' and '```' in the response'''},
        {"role": "user", "content": [
            {
                "type": "text",
                "text": prompt
            }
        ] + [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image}"
                }
            } for image in images
        ]}
    ]

    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        max_tokens=500,
    )
    res = response.choices[0].message.content
    return res

In [7]:
pdf_path = "C:/Users/Sahithi - Rishitha/Downloads/SIH/samples/70__ATS_rating.pdf"
images = convert_pdf_to_images(pdf_path, output_folder=".")
prompt = "get the dopcument type from the image"   
image_details = get_image_details(images, prompt)
image_details

'{\n    "document_type": "resume"\n}'